ACLED Central America data from 01 January 2019 - 07 April 2020
Link to ACLED: https://acleddata.com/#/dashboard 
Information on attribution: https://acleddata.com/acleddatanew/wp-content/uploads/dlm_uploads/2019/07/ACLED_TermsofUseAttributionPolicy_4.2019.pdf

# Data Import and Setup

In [1]:
#Import packages and create dataframe from csv of data
import pandas as pd
df = pd.read_csv(r"sampleACLED.csv")

In [2]:
#retrieve list of attribute names
list(df)

['data_id',
 'iso',
 'event_id_cnty',
 'event_id_no_cnty',
 'event_date',
 'year',
 'time_precision',
 'event_type',
 'sub_event_type',
 'actor1',
 'assoc_actor_1',
 'inter1',
 'actor2',
 'assoc_actor_2',
 'inter2',
 'interaction',
 'region',
 'country',
 'admin1',
 'admin2',
 'admin3',
 'location',
 'latitude',
 'longitude',
 'geo_precision',
 'source',
 'source_scale',
 'notes',
 'fatalities',
 'timestamp',
 'iso3']

In [3]:
#Retrieve count of records in every attribute in the dataframe
df.count()

data_id             368
iso                 368
event_id_cnty       368
event_id_no_cnty    368
event_date          368
year                368
time_precision      368
event_type          368
sub_event_type      368
actor1              368
assoc_actor_1        98
inter1              368
actor2              218
assoc_actor_2        44
inter2              368
interaction         368
region              368
country             368
admin1              368
admin2              360
admin3               39
location            368
latitude            368
longitude           368
geo_precision       368
source              368
source_scale        368
notes               368
fatalities          368
timestamp           368
iso3                368
dtype: int64

# Basic Counts

In [4]:
#Count of events by day
df_dates = df.groupby('event_date')['data_id'].count()
df_dates
#df_dates.to_csv(r'C:/Users/Caitlin/OneDrive/ACLED_CentralAmerica/DataVisFiles/CountOfEventsDay.csv') #export csv from df

event_date
10-Jan-19    26
10-Mar-20     1
10-Nov-19     3
11-Jan-19    31
12-Feb-20     1
12-Jan-19    36
13-Jan-19    44
13-Mar-20     3
13-Nov-19     1
14-Jan-19     4
14-Nov-19     1
15-Mar-20     1
15-Nov-19     4
16-Mar-20     1
16-Nov-19     1
17-Feb-20     2
17-Mar-20     1
18-Feb-20     1
18-Mar-20     2
18-Nov-19     3
19-Feb-20     2
19-Nov-19     1
2-Mar-20      2
20-Feb-20     2
20-Nov-19     2
21-Feb-20     1
21-Mar-20     1
21-Nov-19     9
22-Feb-20     1
22-Nov-19     2
23-Nov-19     2
24-Feb-20     1
27-Jan-20     2
28-Feb-20     1
28-Jan-20     1
29-Feb-20     1
29-Jan-20     2
3-Apr-20      9
3-Feb-20      3
3-Mar-20      4
30-Jan-20     1
4-Apr-20     15
4-Feb-20      3
4-Mar-20      5
5-Feb-20      1
5-Mar-20      1
6-Mar-20      5
7-Feb-20      3
7-Jan-19     36
7-Mar-20      1
8-Jan-19     46
8-Mar-20      3
9-Feb-20      1
9-Jan-19     30
9-Mar-20      1
Name: data_id, dtype: int64

In [5]:
#Count of events by country
df_country = df.groupby('country')['data_id'].count()
df_country
#df_country.to_csv(r'C:/Users/Caitlin/OneDrive/ACLED_CentralAmerica/DataVisFiles/CountPerCountry.csv') #export csv from df

country
Belize           8
Costa Rica       1
El Salvador      7
Guatemala       10
Honduras        15
Mexico         272
Nicaragua       16
Panama          39
Name: data_id, dtype: int64

In [6]:
#count of each event type for entire dataset
df_eventType = df.groupby('event_type')['data_id'].count()
df_eventType

event_type
Battles                        39
Explosions/Remote violence      1
Protests                      142
Riots                          20
Strategic developments         15
Violence against civilians    151
Name: data_id, dtype: int64

In [7]:
#count of each sub event type for entire dataset
df_subEventType = df.groupby('sub_event_type')['data_id'].count()
df_subEventType

sub_event_type
Abduction/forced disappearance        8
Armed clash                          39
Arrests                               2
Attack                              143
Change to group/activity              3
Grenade                               1
Headquarters or base established      1
Looting/property destruction          9
Mob violence                         11
Peaceful protest                    138
Protest with intervention             4
Violent demonstration                 9
Name: data_id, dtype: int64

In [8]:
#find the distribution of events for the types of geo_precision
df_precision = df.groupby('geo_precision')['data_id'].count()
df_precision

geo_precision
1    331
2     32
3      5
Name: data_id, dtype: int64

# Finding Anomalies

In [9]:
#find dates that are not "normal" i.e. are outside of the mean +/- 2 times the standard deviation
#prints a list of dates and number of events
import numpy as np
mean = df_dates.mean()
std = np.std(df_dates)

for count, index in zip(df_dates, df_dates.index):
    # If the value for the date is less than the mean - 2 times the standard deviation or more than the mean + 2 time the std
    if count < (mean - 2*std) or count > (mean + 2*std):
        print(str(index).split(" ")[0], count)

11-Jan-19 31
12-Jan-19 36
13-Jan-19 44
7-Jan-19 36
8-Jan-19 46
9-Jan-19 30


# Geo-Precision Levels Analysis

In [10]:
#subset the geo_precision event records for further analysis 
df_prec1 = df[df.geo_precision == 1]
df_prec2 = df[df.geo_precision == 2]
df_prec3 = df[df.geo_precision == 3]

In [11]:
#create grouped dataframes for each precision level by event type
#this cell creates a table that shows the number of records for each event type for each geo-precision
df_prec1Event = df_prec1.groupby('event_type')['data_id'].count().to_frame(name='count_prec1').reset_index()
df_prec2Event = df_prec2.groupby('event_type')['data_id'].count().to_frame(name='count_prec2').reset_index()
df_prec3Event = df_prec3.groupby('event_type')['data_id'].count().to_frame(name='count_prec3').reset_index()
#merge tables to get count by event type for each spatial precision
join1 = df_prec1Event.merge(df_prec2Event)
join2 = join1.merge(df_prec3Event)
join2
#join2.to_csv(r'C:/Users/Caitlin/OneDrive/ACLED_CentralAmerica/DataVisFiles/PrecCountEvents.csv') #export table to csv

,event_type,count_prec1,count_prec2,count_prec3
0,Protests,136,5,1
1,Strategic developments,10,2,3
2,Violence against civilians,132,18,1


In [12]:
#TO DO
#add a column on the table that calculates the percent of total for each precision level 

In [13]:
#create grouped dataframes for each precision level by country
#this cell creates a table that shows the number of records for each country for each geo-precision
df_prec1Country = df_prec1.groupby('country')['data_id'].count().to_frame(name='count_prec1').reset_index()
df_prec2Country = df_prec2.groupby('country')['data_id'].count().to_frame(name='count_prec2').reset_index()
df_prec3Country = df_prec3.groupby('country')['data_id'].count().to_frame(name='count_prec3').reset_index()
#merge tables to get count by country for each spatial precision
join1 = df_prec1Country.merge(df_prec2Country)
join22 = join1.merge(df_prec3Country)
join22

,country,count_prec1,count_prec2,count_prec3
0,Belize,6,1,1
1,Mexico,238,30,4


In [14]:
#TO DO
#add a column on the table that calculates the percent of total for each precision level 

In [17]:
#Test for loop to create geo-precision tables for each country
master_country_list = []
country_list = df.country.unique()
for country in country_list:
    country_df  = df[df.country==country]
    df_prec1 = country_df[country_df.geo_precision == 1]
    df_prec2 = country_df[country_df.geo_precision == 2]
    df_prec3 = country_df[country_df.geo_precision == 3]
    df_prec1Event = df_prec1.groupby('event_type')['data_id'].count().to_frame(name='count_prec1').reset_index()
    df_prec2Event = df_prec2.groupby('event_type')['data_id'].count().to_frame(name='count_prec2').reset_index()
    df_prec3Event = df_prec3.groupby('event_type')['data_id'].count().to_frame(name='count_prec3').reset_index()
    join1 = df_prec1Event.merge(df_prec2Event)
    join2 = join1.merge(df_prec3Event)
    join2
    master_country_list.append(country_df)

In [18]:
len(master_country_list)

8

In [19]:
print(country_list)

['Costa Rica' 'El Salvador' 'Honduras' 'Mexico' 'Panama' 'Belize'
 'Guatemala' 'Nicaragua']


# Analysis of Null Attributes/Levels of Nulls

In [15]:
df.count()

data_id             21249
iso                 21249
event_id_cnty       21249
event_id_no_cnty    21249
event_date          21249
year                21249
time_precision      21249
event_type          21249
sub_event_type      21249
actor1              21249
assoc_actor_1        6405
inter1              21249
actor2              13273
assoc_actor_2        3170
inter2              21249
interaction         21249
region              21249
country             21249
admin1              21249
admin2              21138
admin3                411
location            21249
latitude            21249
longitude           21249
geo_precision       21249
source              21249
source_scale        21249
notes               21249
fatalities          21249
timestamp           21249
iso3                21249
dtype: int64

In [ ]:
#TO DO
#build chart from counts above
#add attribute to chart with percent of nulls

# Paired Attributes

In [ ]:
#TO DO
#Analysis of paired attributes
#pair one: event type and geo-precision
#pair two: sub-event type and geo-precision
#pair three: event type and sub-event type
#pair four: actor1 and event type
#pair five: actor1 and sub-event type

# By Country Analysis - Mexico

In [16]:
#mexico analysis - create the dataframe
df_mexico = df[df.country=='Mexico']

In [19]:
#Count of events by day in Mexico
df_datesMex = df_mexico.groupby('event_date')['data_id'].count()
#df_datesMex
df_dates.to_csv(r'C:/Users/Caitlin/OneDrive/ACLED_CentralAmerica/DataVisFiles/CountOfEventsDayMexico.csv') #export csv from df
#TO DO
#Find way to calculate which days have no records (maybe use a sort by date)

In [22]:
#find dates that are not "normal" i.e. are outside of the mean +/- 2 times the standard deviation
#prints a list of dates and number of events for Mexico dataset specifically
meanMex = df_datesMex.mean()
stdMex = np.std(df_datesMex)

for count, index in zip(df_datesMex, df_datesMex.index):
    # If the value for the date is less than the mean - 2 times the standard deviation or more than the mean + 2 time the std
    if count < (meanMex - 2*stdMex) or count > (meanMex + 2*std):
        print(str(index).split(" ")[0], count)

1-May-19 72
10-Apr-19 84
18-Feb-20 68
21-Dec-19 11
8-Aug-19 78
8-Feb-19 108
8-Mar-20 95


In [20]:
#count of each event type for Mexico
df_eventTypeMex = df_mexico.groupby('event_type')['data_id'].count()
df_eventTypeMex
#TO DO
#Add column with percent of Mexico dataset

event_type
Battles                       1887
Explosions/Remote violence      16
Protests                      6163
Riots                         1002
Strategic developments         493
Violence against civilians    7204
Name: data_id, dtype: int64

In [21]:
#count of each sub event type for Mexico
df_subEventTypeMex = df_mexico.groupby('sub_event_type')['data_id'].count()
df_subEventTypeMex
#TO DO
#Add column with percent of Mexico dataset

sub_event_type
Abduction/forced disappearance         277
Armed clash                           1887
Arrests                                173
Attack                                6865
Change to group/activity                68
Disrupted weapons use                   12
Excessive force against protesters       5
Grenade                                  6
Headquarters or base established         3
Looting/property destruction           160
Mob violence                           659
Other                                   77
Peaceful protest                      6050
Protest with intervention              108
Remote explosive/landmine/IED           10
Sexual violence                         62
Violent demonstration                  343
Name: data_id, dtype: int64